### Text Processing for Author Recognition using Spark

#### Import statements

In [1]:
import pyspark as ps    # import the spark suite
import warnings         # display warning if spark context already exists
import os

#### Initialized Spark Context

In [2]:
try:
    sc = ps.SparkContext('local[4]')                # create spark context to work locally on all available cpus
    print('created SparkContext')
except ValueError:
    warnings.warn('SparkContext already exists')    # issue a warning if context already exists

/Users/ophidian/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: UserWarning: SparkContext already exists


### Read data.json into Spark SQL context

In [3]:
data_file = 'data/data.json'
data = sc.read.json(data_file)

AttributeError: 'SparkContext' object has no attribute 'read'

? need sql context not SparkContext

for fun use RDD and map reduce to remove the double bars I put in!